In [19]:
%pip install pysus pandas pyarrow boto3 python-dotenv

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os, sys
from dotenv import load_dotenv
import pandas as pd
import os
import boto3
from botocore.client import Config

env_path = os.path.join(os.getcwd(), os.pardir, ".env")
load_dotenv(dotenv_path=env_path, override=True)

os.environ["MINIO_ENDPOINT"] = "http://localhost:9000"


print("ENDPOINT:", os.getenv("MINIO_ENDPOINT"))
print("ACCESS_KEY:", os.getenv("MINIO_ACCESS_KEY"))
print("SECRET_KEY:", os.getenv("MINIO_SECRET_KEY"))
print("BUCKET:", os.getenv("MINIO_BUCKET"))
print("PREFIX:", os.getenv("MINIO_S3_PREFIX"))

ENDPOINT: http://localhost:9000
ACCESS_KEY: KJzdcNsBU1WEDCVlCoes
SECRET_KEY: alsWZqBJeEqCeDJ9iq53lWwLwC8Oncfux5Z08mgv
BUCKET: davintwarehouse
PREFIX: sih/


In [21]:
from minio_loader import load_sih_dataframe

df = load_sih_dataframe()
df

🔄 Carregando sih/RDSP2001.parquet...
🔄 Carregando sih/RDSP2002.parquet...
🔄 Carregando sih/RDSP2003.parquet...


,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,NASC,...,DIAGSEC9,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7,TPDISEC8,TPDISEC9
0,350000,2020,01,01,47074851000819,3520100127380,1,15950000,354560,19990708,...,,1,0,0,0,0,0,0,0,0
1,350000,2020,01,01,47074851000819,3520100127401,1,15940000,351560,19780313,...,,1,1,0,0,0,0,0,0,0
2,350000,2020,01,01,47074851000819,3520100127478,1,15828000,353510,19870228,...,,1,1,0,0,0,0,0,0,0
3,350000,2020,01,01,47074851000819,3520100127500,1,15813283,351110,19640504,...,,1,0,0,0,0,0,0,0,0
4,350000,2020,01,01,47074851000819,3520100127511,1,14960000,353350,19671209,...,,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630033,355030,2020,03,02,46392148001272,3520108546196,1,05264050,355030,19830507,...,,0,0,0,0,0,0,0,0,0
630034,355030,2020,03,02,46392148001272,3520108546218,1,05135380,355030,19930606,...,,0,0,0,0,0,0,0,0,0
630035,355030,2020,03,02,46392148001272,3520108546229,1,05265050,355030,19870128,...,,0,0,0,0,0,0,0,0,0
630036,355030,2020,03,02,46392148001272,3520108546240,1,05161105,355030,19910329,...,,0,0,0,0,0,0,0,0,0


In [22]:
df[['DT_INTER', 'DT_SAIDA']].head()

,DT_INTER,DT_SAIDA
0,20200119,20200121
1,20200119,20200122
2,20200117,20200120
3,20200118,20200120
4,20200118,20200121


In [23]:
print(f"Dimensões do DataFrame: {df.shape}")

Dimensões do DataFrame: (630038, 113)


In [15]:
colunas_uteis = [
    'N_AIH', 'IDENT', 'CEP', 'MUNIC_RES', 'NASC', 'SEXO',
    'UTI_MES_IN', 'UTI_MES_AN', 'MARCA_UTI', 'DIAR_UTI',
    'DT_INTER', 'DT_SAIDA', 'DIAG_PRINC', 'DIAG_SECUN',
    'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_TOT', 'VAL_UTI',
    'DIAS_PERM', 'MORTE', 'IDADE', 'CNES', 'COMPLEX', 'GESTAO'
]

In [24]:
colunas_existentes = [col for col in colunas_uteis if col in df.columns]
df_analise = df[colunas_existentes].copy()

# 4. datas para datetime
if 'DT_INTER' in df_analise.columns:
    df_analise['DT_INTER'] = pd.to_datetime(df_analise['DT_INTER'], errors='coerce')
    df_analise['MES'] = df_analise['DT_INTER'].dt.month
    df_analise['ANO'] = df_analise['DT_INTER'].dt.year

if 'DT_SAIDA' in df_analise.columns:
    df_analise['DT_SAIDA'] = pd.to_datetime(df_analise['DT_SAIDA'], errors='coerce')

df_analise

,N_AIH,IDENT,CEP,MUNIC_RES,NASC,SEXO,UTI_MES_IN,UTI_MES_AN,MARCA_UTI,DT_INTER,...,VAL_TOT,VAL_UTI,DIAS_PERM,MORTE,IDADE,CNES,COMPLEX,GESTAO,MES,ANO
0,3520100127380,1,15950000,354560,19990708,1,0,0,00,2020-01-19,...,1525.93,0.00,2,0,20,2089327,02,2,1,2020
1,3520100127401,1,15940000,351560,19780313,3,0,0,00,2020-01-19,...,901.60,0.00,3,0,41,2089327,02,2,1,2020
2,3520100127478,1,15828000,353510,19870228,3,0,0,00,2020-01-17,...,755.00,0.00,3,0,32,2089327,02,2,1,2020
3,3520100127500,1,15813283,351110,19640504,3,0,0,00,2020-01-18,...,456.67,0.00,2,0,55,2089327,02,2,1,2020
4,3520100127511,1,14960000,353350,19671209,1,0,0,00,2020-01-18,...,2256.85,0.00,3,0,52,2089327,02,2,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630033,3520108546196,1,05264050,355030,19830507,3,0,0,00,2020-03-12,...,514.60,0.00,2,0,36,2077450,02,1,3,2020
630034,3520108546218,1,05135380,355030,19930606,3,0,0,00,2020-03-12,...,549.60,0.00,2,0,26,2077450,02,1,3,2020
630035,3520108546229,1,05265050,355030,19870128,3,0,0,00,2020-03-09,...,718.11,0.00,3,0,33,2077450,02,1,3,2020
630036,3520108546240,1,05161105,355030,19910329,3,0,0,00,2020-03-10,...,678.11,0.00,3,0,28,2077450,02,1,3,2020


In [25]:
# --- 1) Reuso das configs do .env
ENDPOINT   = os.environ["MINIO_ENDPOINT"]
ACCESS_KEY = os.environ["MINIO_ACCESS_KEY"]
SECRET_KEY = os.environ["MINIO_SECRET_KEY"]
REGION     = os.getenv("AWS_REGION", "us-east-1")

# --- 2) Parâmetros para output ---
OUTPUT_BUCKET = os.getenv("MINIO_OUTPUT_BUCKET", "analise")
OUTPUT_KEY    = os.getenv("MINIO_OUTPUT_KEY", "df_analise.csv")

In [27]:
from utils.minio_utils import criar_bucket, upload_csv

bucket_name = "bucket-teste"
criar_bucket(bucket_name)

upload_csv(df_analise, bucket_name, "sih_analise_dados.csv")

Bucket 'bucket-teste' não existe. Criando...
Bucket 'bucket-teste' criado com sucesso.
📤 Enviando sih_analise_dados.csv para s3://bucket-teste/sih_analise_dados.csv
Upload concluído com sucesso!


True